In [ ]:
import numpy as np
import pymoo
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from cvxopt import matrix
from cvxopt.solvers import qp
import matplotlib.pyplot as plt
from trieste.experimental.qhsri.qhsri import BatchHypervolumeSharpeRatioIndicator

# Trying it out

In [ ]:
from trieste.objectives import scaled_branin
from trieste.objectives.utils import mk_observer
from trieste.space import Box

# Create the observer
observer = mk_observer(scaled_branin)

# Define Search space
search_space = Box([0,0], [1,1])

# Set initial number of query points
num_initial_points = 5
initial_query_points = search_space.sample(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
from trieste.models.gpflow import GaussianProcessRegression, build_gpr

# Set up model
gpflow_model = build_gpr(initial_data, search_space, likelihood_variance=1e-7)
model = GaussianProcessRegression(gpflow_model)

In [ ]:
# Set up rule

# Get current best y
current_best = float(min(initial_data.observations))

qhsri = BatchHypervolumeSharpeRatioIndicator(model, current_best)

points_chosen_by_qhsri = qhsri(model, 5)


In [ ]:
from trieste.acquisition.function import BatchMonteCarloExpectedImprovement
from trieste.acquisition.rule import EfficientGlobalOptimization


# Batch MCEI requires a number of monte carlo samples to do
monte_carlo_sample_size = 1000
# Set up batch MCIE builder
batch_ei_acq = BatchMonteCarloExpectedImprovement(
    sample_size=monte_carlo_sample_size,
    jitter = 1e-5
)
# Set up batch EGO acquisition rule
batch_size = 5
batch_ei_acq_rule = EfficientGlobalOptimization(
    num_query_points=batch_size,
     builder=batch_ei_acq)

# Use acquisition rule to pick new points
points_chosen_by_batch_ei = np.array(batch_ei_acq_rule.acquire_single(search_space, model, dataset=initial_data))

In [ ]:
import matplotlib.pyplot as plt
from trieste.experimental.plotting import plot_acq_function_2d

from trieste.acquisition.function import ExpectedImprovement

# plot standard EI acquisition function
ei = ExpectedImprovement()
ei_acq_function = ei.prepare_acquisition_function(model, dataset=initial_data)
plot_acq_function_2d(
    ei_acq_function, [0, 0], [1, 1], contour=True, grid_density=100
)

plt.scatter(
    points_chosen_by_batch_ei[:, 0],
    points_chosen_by_batch_ei[:, 1],
    color="red",
    lw=5,
    label="Batch-EI",
    marker="*",
    zorder=1,
)
plt.scatter(
    points_chosen_by_qhsri[:, 0],
    points_chosen_by_qhsri[:, 1],
    color="black",
    lw=10,
    label="Local \nPenalization",
    marker="+",
)
plt.show()


In [ ]:
plt.scatter(
    points_chosen_by_batch_ei[:, 0],
    points_chosen_by_batch_ei[:, 1],
    color="red",
    lw=5,
    label="Batch-EI",
    marker="*",
    zorder=1,
)
plt.scatter(
    points_chosen_by_qhsri[:, 0],
    points_chosen_by_qhsri[:, 1],
    color="black",
    lw=10,
    label="Local \nPenalization",
    marker="+",
)
plt.show()
